In [4]:
pwd()
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ
readdir()

  Activating project at `~/code/FermiCG`


36-element Vector{String}:
 ".ipynb_checkpoints"
 "C_act.molden"
 "C_act_loc.molden"
 "C_act_loc_sorted.molden"
 "C_act_loc_sorted_2.molden"
 "C_act_loc_sorted_3.molden"
 "C_doc.molden"
 "README.md"
 "b3lyp-d3_def2svp.xyz"
 "b3lyp-d3_def2svp_S4.xyz"
 "clustered1_integrals_h0.npy"
 "clustered1_integrals_h1.npy"
 "clustered1_integrals_h2.npy"
 ⋮
 "cr_qubit_scf.ipynb"
 "cr_qubit_scf_integrals_h0.npz.npy"
 "cr_qubit_scf_integrals_h1.npz.npy"
 "cr_qubit_scf_integrals_h2.npz.npy"
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "scf.ipynb"
 "uhf_density.npy"
 "uhf_mo_coeffs_a.npy"
 "uhf_mo_coeffs_b.npy"
 "venv"
 "wb97x-v_def2svp.xyz"

In [5]:
using FermiCG, NPZ, JLD2


In [6]:
#load integrals from disk
h0 = npzread("clustered1_integrals_h0.npy")
h1 = npzread("clustered1_integrals_h1.npy")
h2 = npzread("clustered1_integrals_h2.npy")
ints = InCoreInts(h0, h1, h2)
ints_original = deepcopy(ints);

println(" Integrals have the following sizes: h0= ", size(h0), " h1= ", size(h1), " h2= ", size(h2))


# Define clusters - this should probably be done in the python notebook, and then just read in here
clusters_in = [
	       (1:6), # Benzene 1
	       (7:12), # Benzene 1
	       (13:18), # Benzene 1
	       (19:24), # Benzene 1
	       (25:26),   # metal 4s (sig,sig*)
	       (27:34),   # metal 3d + sig
	       ]

clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]

init_fspace = [
	       (3,3),
	       (3,3),
	       (3,3),
	       (3,3),
	       (1,1),
	       (4,4)];
display(clusters)
display(init_fspace)

6-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5, 6])
 Cluster(2, [7, 8, 9, 10, 11, 12])
 Cluster(3, [13, 14, 15, 16, 17, 18])
 Cluster(4, [19, 20, 21, 22, 23, 24])
 Cluster(5, [25, 26])
 Cluster(6, [27, 28, 29, 30, 31, 32, 33, 34])

6-element Vector{Tuple{Int64, Int64}}:
 (3, 3)
 (3, 3)
 (3, 3)
 (3, 3)
 (1, 1)
 (4, 4)

 Integrals have the following sizes: h0= () h1= (34, 34) h2= (34, 34, 34, 34)


In [8]:
rdm1 = zeros(size(ints.h1))

e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, verbose=0, gconv=1e-6, method="bfgs", sequential=true);

ints = FermiCG.orbital_rotation(ints_original,U);

@save "cmf_data_2.jld2" ints clusters init_fspace Da Db

 ooCMF Iter:    1 Total= -2119.115642802029 Active= -105.855805529394 G=     3.38e-01
 ooCMF Iter:    2 Total= -2119.153672406961 Active= -105.893835134326 G=     9.12e-02
 ooCMF Iter:    3 Total= -2119.156237385084 Active= -105.896400112449 G=     4.66e-02
 ooCMF Iter:    4 Total= -2119.157060643232 Active= -105.897223370597 G=     3.35e-02
 ooCMF Iter:    5 Total= -2119.157531350253 Active= -105.897694077617 G=     3.14e-02
 ooCMF Iter:    6 Total= -2119.158013999991 Active= -105.898176727355 G=     2.23e-02
 ooCMF Iter:    7 Total= -2119.158144705245 Active= -105.898307432610 G=     8.93e-03
 ooCMF Iter:    8 Total= -2119.158165764305 Active= -105.898328491670 G=     3.93e-03
 ooCMF Iter:    9 Total= -2119.158171846771 Active= -105.898334574136 G=     3.14e-03
 ooCMF Iter:   10 Total= -2119.158178342208 Active= -105.898341069573 G=     2.70e-03
 ooCMF Iter:   11 Total= -2119.158181627153 Active= -105.898344354518 G=     1.57e-03
 ooCMF Iter:   12 Total= -2119.158182395530 Active= -1

 * Status: success

 * Candidate solution
    Final objective value:     -2.119158e+03

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 2.50e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.54e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.68e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.27e-14 ≰ 0.0e+00
    |g(x)|                 = 8.97e-07 ≤ 1.0e-06

 * Work counters
    Seconds run:   128  (vs limit Inf)
    Iterations:    28
    f(x) calls:    65
    ∇f(x) calls:   65


 ooCMF Iter:   29 Total= -2119.158183096122 Active= -105.898345823487 G=     4.45e-06
*ooCMF -2119.15818310 
